In [ ]:
!wget http://snap.stanford.edu/data/amazon/productGraph/categoryFiles/reviews_Cell_Phones_and_Accessories_5.json.gz

--2020-12-12 10:54:33--  http://snap.stanford.edu/data/amazon/productGraph/categoryFiles/reviews_Cell_Phones_and_Accessories_5.json.gz
Resolving snap.stanford.edu (snap.stanford.edu)... 171.64.75.80
Connecting to snap.stanford.edu (snap.stanford.edu)|171.64.75.80|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 45409631 (43M) [application/x-gzip]
Saving to: ‘reviews_Cell_Phones_and_Accessories_5.json.gz’

reviews_Cell_Phones 100%[===================>]  43.31M  31.9MB/s    in 1.4s    

2020-12-12 10:54:35 (31.9 MB/s) - ‘reviews_Cell_Phones_and_Accessories_5.json.gz’ saved [45409631/45409631]



In [ ]:
!python3 -m nltk.downloader stopwords
!python3 -m nltk.downloader universal_tagset
!python3 -m spacy download en # download the english model

In [ ]:
!pip install git+https://github.com/boudinfl/pke.git

In [ ]:
!pip install flair

In [ ]:
from flair.embeddings import WordEmbeddings, FlairEmbeddings, StackedEmbeddings

# create a StackedEmbedding object that combines glove and forward/backward flair embeddings
stacked_embeddings = StackedEmbeddings([
                                        WordEmbeddings('glove'),
                                        FlairEmbeddings('news-forward'),
                                        FlairEmbeddings('news-backward'),
                                       ])

In [218]:
we = {}

In [219]:
from flair.data import Sentence
sentence = Sentence(s)

# just embed a sentence using the StackedEmbedding as you would with any single embedding.
stacked_embeddings.embed(sentence)

# now check out the embedded tokens.
for token in sentence:
    we[token.text] = token.embedding
    # print(token)
    # print(token.embedding)

In [217]:
with open('some.txt', 'r') as f:
    s = f.read()

In [235]:
import pke

# define the valid Part-of-Speeches to occur in the graph
pos = {'NOUN', 'PROPN', 'ADJ'}

# define the grammar for selecting the keyphrase candidates
grammar = "NP: {<ADJ>*<NOUN|PROPN>+}"

# 1. create a PositionRank extractor.
extractor = pke.unsupervised.PositionRank()

# 2. load the content of the document.
extractor.load_document(input='some.txt',
                        language='en',
                        normalization=None)

# 3. select the noun phrases up to 3 words as keyphrase candidates.
extractor.candidate_selection(grammar=grammar,
                              maximum_word_number=3)

# 4. weight the candidates using the sum of their word's scores that are
#    computed using random walk biaised with the position of the words
#    in the document. In the graph, nodes are words (nouns and
#    adjectives only) that are connected if they occur in a window of
#    10 words.
extractor.candidate_weighting(window=10,
                              pos=pos)

# 5. get the 10-highest scored candidates as keyphrases
keyphrases = extractor.get_n_best(n=20)

In [236]:
keyphrases

[('nokia n95', 0.06092104863457341),
 ('gb phone', 0.05412068928870925),
 ('symbian phone os', 0.04900936897837181),
 ('other phone', 0.04866889217405842),
 ('screen phone', 0.048422256970365785),
 ('symbian s60 phone', 0.04750457766834135),
 ('andoff speaker phone', 0.046174640038317435),
 ('excellent phone', 0.04375042522100936),
 ('good phone', 0.04286974393460123),
 ('smart phone', 0.04178830538210883),
 ('cell phone towers', 0.0416999588411568),
 ('phone orientation', 0.04138319162742759),
 ('end phone', 0.040926074439931605),
 ('phone', 0.04040566309507548),
 ('n95 nam', 0.038142730814062095),
 ('n95 vga', 0.03687641064014312),
 ('original n95', 0.036875806449055824),
 ('vivid n95', 0.036148295563029294),
 ('n95', 0.03557082701429333),
 ('other nokia phones==========================since', 0.034480141407807925)]

In [220]:
we['great'].numpy().shape

(4196,)

In [221]:
import torch
def get_mean_emb(phrase):
    ws = phrase.split(" ")
    vect = torch.zeros(4196)
    c = 1
    for i in ws:
        try:
            vect += we[i]
            c += 1
        except KeyError:
            pass
    return (vect/c).numpy()

In [222]:
from sklearn.cluster import KMeans
import numpy as np
X = np.array([   get_mean_emb(i) for i,j in keyphrases   ])

kmeans = KMeans(n_clusters=8, random_state=0).fit(X)
kmeans.labels_
# kmeans.cluster_centers_

array([5, 5, 2, 1, 1, 2, 1, 1, 1, 1, 1, 3, 0, 4, 3, 2, 2, 7, 3, 6],
      dtype=int32)

In [223]:
p = np.array(keyphrases)

In [224]:
for i in range(10):
    print(p[kmeans.labels_ == i])
    print("  ")

[['phone end' '0.04080380660812189']]
  
[['long cord' '0.05929613746987916']
 ['longer cord' '0.05445652337418146']
 ['usual cord' '0.04696344713663413']
 ['shorter cord' '0.04671614980877882']
 ['generic cord' '0.045806561723050476']
 ['cord break' '0.04534207223378428']
 ['cord' '0.043751269015709235']]
  
[['iphone cable' '0.06551509301320887']
 ['long iphone' '0.05037817301828792']
 ['older iphone' '0.03652705073786119']
 ['iphone' '0.03483330456411799']]
  
[['longer cable' '0.04138704280756311']
 ['oem apple cable' '0.03712914610527167']
 ['sync cable' '0.032498264040691734']]
  
[['ipad' '0.03808949425186152']]
  
[['cable works' '0.1031432772035251']
 ['works' '0.07246148875443421']]
  
[['2nd gen ipod' '0.03249678586160086']]
  
[['ipod touch' '0.03451572354825621']]
  
[]
  
[]
  


In [170]:
"Over the years, my household has tried multiple Bluetooth solutions for our hands-free communications needs. We\'ve had acceptable results from a solution integrated into GPS...but much of our regular driving doesn\'t include the need to load up a full-size GPS. We\'ve".replace("\'", "'")

"Over the years, my household has tried multiple Bluetooth solutions for our hands-free communications needs. We've had acceptable results from a solution integrated into GPS...but much of our regular driving doesn't include the need to load up a full-size GPS. We've"

In [234]:
myString = ". ".join(df[df['asin'] == "B0014KLFN6"]['reviewText'])
with open('some.txt','w') as f:
    f.write(myString)
myString

'I am very happy with my Nokia N95 8GB phone.As the title says MORE THEN I EXPECTED!I would like to take a moment to thank Amazon forquick and courteous service.OK. My review...First of all the phone itself is excellent. both on andoff speaker phone. It is loud and clear. sometimes evenbetter quality then a land line.Second is the music and video players. The player thatcomes with the phone is great but, if you want morediversity with different codec\'s try Core video player.You can find it on most symbian forums.People compare this phone to the iphone and, thats fine.I\'ve owned both now and, have to say that there aremore application and other items you can get on the N95then the iphone...and as far as I\'m concerned the callquality is so much better with the Nokia.On a scale of 1 to 10 I would rate this phone a solid 9 becauseof it\'s quirks. Being able to charge via usb is one and, withmost smart phone an occasional unscheduled reboot.But, even with it\'s minor drawbacks its a phon

In [233]:
df['asin'].unique()[334]

'B0014KLFN6'

In [ ]:
import pandas as pd
import gzip

def parse(path):
  g = gzip.open(path, 'rb')
  for l in g:
    yield eval(l)

def getDF(path):
  i = 0
  df = {}
  for d in parse(path):
    df[i] = d
    i += 1
  return pd.DataFrame.from_dict(df, orient='index')

df = getDF('reviews_Cell_Phones_and_Accessories_5.json.gz')

In [ ]:
from flair.embeddings import TransformerDocumentEmbeddings

# init embedding
embedding = TransformerDocumentEmbeddings('roberta-base')

# create a sentence
sentence = Sentence('The grass is green .')

# embed the sentence
embedding.embed(sentence)

[Sentence: "The grass is green ."   [− Tokens: 5]]

In [ ]:
print(sentence.embedding)

tensor([-4.7716e-02,  7.4434e-02, -5.7382e-02, -1.1084e-01,  9.3728e-02,
        -7.0667e-02, -3.9124e-02, -1.5208e-02,  8.2914e-03, -6.2067e-02,
         1.3514e-02,  5.5369e-02,  3.6431e-02, -4.2434e-02,  6.5581e-02,
         6.7928e-02,  4.3899e-02,  1.2971e-02, -2.2806e-02,  2.6699e-02,
        -1.2290e-01,  8.4062e-02, -1.1518e-02,  6.3545e-02,  8.2313e-02,
         5.8250e-02,  6.7556e-02,  5.4044e-02, -1.5503e-02, -2.3941e-02,
        -7.6196e-02, -4.3455e-03,  4.6730e-02, -3.2246e-02,  6.4714e-02,
         7.0874e-02,  3.7649e-02, -4.6265e-02, -1.3970e-01,  1.8995e-02,
        -5.0075e-02,  3.4664e-02,  9.9441e-02, -3.2781e-02,  1.2399e-01,
         8.9412e-03,  8.6673e-03,  4.7400e-02, -2.7428e-02,  1.9678e-02,
         4.4378e-03,  8.6579e-03, -4.7181e-02, -9.1996e-02, -7.2369e-02,
         5.5046e-02,  8.5535e-02,  7.4627e-02,  1.1019e-01, -2.0454e-02,
         2.6471e-03, -1.2888e-01, -8.9457e-02, -3.1360e-02,  3.5597e-02,
        -1.1634e-02,  5.0005e-02,  6.4393e-04,  4.1